# Network analysis with NetworkX

In [37]:
import os, sys
import pandas as pd
import networkx as nx

from globes import taxi_dir, days_dir, getFullDf, zoneIdToBorough

In [38]:
ZONE_COLS = [
    "pickup_day",
    "pickup_hour",
    "pickup_zone_taxi", 
    "dropoff_zone_taxi", 
    "pickup_borough", 
    "dropoff_borough"
]

zone_nodes = [i for i in range(1, 264) if i not in [57, 104, 105]]

df = getFullDf()
df = df[ZONE_COLS]

using 28 cores


In [39]:
## functions for loading graphs
def loadNodes(G):
    # load nodes
    BoroughDict = zoneIdToBorough()
    nodes = {}
    for zoneid in zone_nodes:
        borough = BoroughDict.get(zoneid) if BoroughDict.get(zoneid) else "None"
        if nodes.get(borough):
            nodes[borough].append(zoneid)
        else:
            nodes[borough] = [zoneid]
    
    for borough in nodes:
        G.add_nodes_from(nodes[borough], borough=borough)
    
    return G
        
def loadDirectedEdges(G):
    edges = []
    for pickup_zone in zone_nodes:
        df_zone = df.loc[df["pickup_zone_taxi"] == pickup_zone]
        pickup_group = df_zone.groupby("dropoff_zone_taxi").size()
        for dropoff_zone, count in pickup_group.iteritems():
            edges.append((pickup_zone, dropoff_zone, count))
            
    G.add_weighted_edges_from(edges)
    return G

def loadUndirectedEdges(G):
    edgeDict = {}
    # load edgeDict: (smaller node, bigger node) => agg flow
    for pickup_zone in zone_nodes:
        df_zone = df.loc[df["pickup_zone_taxi"] == pickup_zone]
        pickup_group = df_zone.groupby("dropoff_zone_taxi").size()
        for dropoff_zone, count in pickup_group.iteritems():
            edge = (min(dropoff_zone, pickup_zone), max(dropoff_zone, pickup_zone))
            if edge in edgeDict:
                edgeDict[edge] += count
            else:
                edgeDict[edge] = count

    edges = [(e1, e2, edgeDict[(e1, e2)]) for e1,e2 in edgeDict]
    G.add_weighted_edges_from(edges)
    return G


In [42]:
## load digraph
G = nx.MultiDiGraph()
G = loadNodes(G)
G = loadDirectedEdges(G)

## undirected graph, sum of flow between 
G_aggflow = nx.MultiGraph()
G_aggflow = loadNodes(G_aggflow)
G_aggflow = loadUndirectedEdges(G_aggflow)

print len(G.nodes())
print len(G_aggflow.nodes())

260
260


## Measures of centrality

In [43]:
def boroughAgg(G, centrality_data):
    agg = {}
    for zoneid in centrality_data:
        borough = G.node[zoneid]["borough"]
        if not borough in agg:
            agg[borough] = [centrality_data[zoneid]]
        else:
            agg[borough].append(centrality_data[zoneid])
    
    # get means for each borough
    for borough in agg:
        agg[borough] = sum(agg[borough])/len(agg[borough])
    return agg


In [44]:
deg_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

print boroughAgg(G, deg_centrality)
print boroughAgg(G, closeness_centrality)
print boroughAgg(G, betweenness_centrality)

{u'Staten Island': 0.45598455598455595, u'Brooklyn': 1.4281916576998546, u'Bronx': 1.1753614079195474, u'EWR': 0.8918918918918919, u'Manhattan': 1.6707773871952976, u'Queens': 1.2869634340222578}
{u'Staten Island': 0.4881828746417568, u'Brooklyn': 0.8014336481213263, u'Bronx': 0.7003357195386488, u'EWR': 0.5365425073149499, u'Manhattan': 0.9310309335791058, u'Queens': 0.7462524180823938}
{u'Staten Island': 0.0004682492370728995, u'Brooklyn': 0.0013879368041661217, u'Bronx': 0.000454884920034402, u'EWR': 0.00023518068684266527, u'Manhattan': 0.0022433698782293667, u'Queens': 0.0010449833194460766}


In [45]:
deg_centrality = nx.degree_centrality(G_aggflow)
closeness_centrality = nx.closeness_centrality(G_aggflow)
betweenness_centrality = nx.betweenness_centrality(G_aggflow)

print boroughAgg(G_aggflow, deg_centrality)
print boroughAgg(G_aggflow, closeness_centrality)
print boroughAgg(G_aggflow, betweenness_centrality)

{u'Staten Island': 0.38687258687258697, u'Brooklyn': 0.8141654535097158, u'Bronx': 0.6866301517464307, u'EWR': 0.7490347490347491, u'Manhattan': 0.9298104074223474, u'Queens': 0.7621508062684536}
{u'Staten Island': 0.5952008448493189, u'Brooklyn': 0.8454540651787196, u'Bronx': 0.7621253250993933, u'EWR': 0.7932217932217932, u'Manhattan': 0.9380565641375687, u'Queens': 0.8152363495637065}
{u'Staten Island': 0.00014428834231732475, u'Brooklyn': 0.0008336645781396832, u'Bronx': 0.0002710839050134295, u'EWR': 0.00037929385337888546, u'Manhattan': 0.0017771252999865773, u'Queens': 0.0005991439028553201}


In [46]:
cliques = list(nx.find_cliques(G_aggflow))
print cliques

In [47]:
cliques

[[230,
  132,
  48,
  50,
  68,
  79,
  100,
  162,
  164,
  170,
  59,
  4,
  7,
  24,
  25,
  32,
  41,
  42,
  43,
  47,
  69,
  74,
  75,
  83,
  88,
  90,
  92,
  97,
  262,
  263,
  137,
  145,
  138,
  140,
  141,
  236,
  142,
  143,
  144,
  151,
  152,
  158,
  161,
  116,
  163,
  166,
  167,
  113,
  159,
  168,
  182,
  114,
  186,
  119,
  249,
  220,
  223,
  229,
  234,
  235,
  237,
  238,
  239,
  243,
  244,
  126,
  246,
  247,
  148,
  125,
  255,
  256,
  80,
  184,
  259,
  18,
  169,
  185,
  208,
  248,
  212,
  213,
  224,
  127,
  240,
  51,
  136,
  94,
  174,
  78,
  241,
  242,
  147,
  20,
  183,
  120,
  250,
  60,
  254,
  31],
 [230,
  132,
  48,
  50,
  68,
  79,
  100,
  162,
  164,
  170,
  59,
  4,
  7,
  24,
  25,
  32,
  41,
  42,
  43,
  47,
  69,
  74,
  75,
  83,
  88,
  90,
  92,
  97,
  262,
  263,
  137,
  145,
  138,
  140,
  141,
  236,
  142,
  143,
  144,
  151,
  152,
  158,
  161,
  116,
  163,
  166,
  167,
  113,
  159,
  168,
  182